In [ ]:
import logging
import sys
import pandas as pd

# to include the SR code without installing in the environment
sys.path.append('../')

from symbolic_regression.SymbolicRegressor import SymbolicRegressor

# The operations

Here we define the list of allowed operations. In this project we implemented most of the arithmetic operations we expect to need in a normal use. Please have a look at the file in `symbolic_regression/operators.py` to see how we define them and to define your own operators.

In [ ]:
from symbolic_regression.operators import *

operations = [
    OPERATOR_ADD,
    OPERATOR_SUB,
    OPERATOR_MUL,
    OPERATOR_DIV,
    # OPERATOR_ABS,
    # OPERATOR_MOD,
    # OPERATOR_NEG,
    # OPERATOR_INV,
    OPERATOR_LOG,
    OPERATOR_EXP,
    OPERATOR_POW,
    OPERATOR_SQRT,
    OPERATOR_MAX,
    OPERATOR_MIN
]


# The example dataset: Body Fat Index

This is the generation of a score to predict the Body Fat Intex of a person.

In [ ]:
def min_max_normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (
            df[feature_name] - min_value) / (max_value - min_value)
    return result


def std_normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        mean_val = df[feature_name].mean()
        std_val = df[feature_name].std()
        result[feature_name] = (df[feature_name] - mean_val) / std_val
    return result


In [ ]:
data = pd.read_csv(f'./body_fat.csv')
data = data.drop(41)  # Drop the outlier

In [ ]:
# Data engineering and normalization
Weight_lb_to_kg = data['Weight']*0.453592
Height_inches_to_m = data['Height']*0.0254
BMI = Weight_lb_to_kg/(Height_inches_to_m**2)

proxy = (BMI-BMI.mean())/BMI.std()

data = std_normalize(data)

bins = 16

features = list(data.columns)
features.remove('BodyFat')
features.remove('Density')
target = 'BodyFat'
weights = 'w'

data['BodyFat_bin'] = pd.qcut(data[target], 10, labels=False).astype('int')

from symbolic_regression.multiobjective.fitness.DistributionPreserving import get_cumulant_hist
from symbolic_regression.multiobjective.fitness.Regression import create_regression_weights

F_y=get_cumulant_hist(data=data,target=target,bins=bins)
data[weights]=create_regression_weights(data=data,target=target,bins=bins)

print(f'Dataset {data.shape}')


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[features+[weights]], data[target], test_size=0.2, random_state=42, stratify=data['BodyFat_bin'])

# Unify features and target in a single dataframe
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

print(f'Train {train.shape}')
print(f'Test {test.shape}')

Here we define the base range for which to generate the constants in the individuals. Furthermore, we also define how to optimize those constants in order to make them converge to the best value they can have in their expression.

We are using ADAM with the following configuration parameters.

In [ ]:
const_range = (0, 1)

constants_optimization = 'scipy'
constants_optimization_conf = {'task': 'regression:wmse'}


In [ ]:
from symbolic_regression.multiobjective.fitness.DistributionPreserving import Wasserstein
from symbolic_regression.multiobjective.fitness.Correlation import KendallTauCorrelation
from symbolic_regression.multiobjective.fitness.Regression import WeightedMeanSquaredError

fitness_functions = [
    WeightedMeanSquaredError(label='wmse', target=target,
                             weights=weights, minimize=True, hypervolume_reference=data[target].abs().max(), 
                             constants_optimization=constants_optimization, 
                             constants_optimization_conf=constants_optimization_conf),
    KendallTauCorrelation(label='1-kendalltau', target=target,
                          one_minus=True, minimize=True, hypervolume_reference=1.1),
    Wasserstein(label='wasserstein', target=target, weights=weights, F_y=F_y,
                bins=10, minimize=True, hypervolume_reference=1.1)
]


''' Use this to modulate the relative frequency of genetic operations
    E.g., crossover is chosen 2 times more frequently than mutation
        {
            'crossover': 2,
            'mutation': 1,
            # etc...
        }
'''
genetic_operators_frequency = {
    'crossover': 1,
    'mutation': 1,
    'insert_node': 1,
    'delete_node': 1,
    'mutate_leaf': 1,
    'mutate_operator': 1,
    'recalibrate': 1
}


In [ ]:
from symbolic_regression.callbacks.CallbackSave import MOSRCallbackSaveCheckpoint
from symbolic_regression.callbacks.CallbackStatistics import MOSRHistory, MOSRStatisticsComputation

file_name = f'./body_fat'

callbacks = [
    MOSRCallbackSaveCheckpoint(
        checkpoint_file=file_name, checkpoint_frequency=1, checkpoint_overwrite=True),
    MOSRStatisticsComputation(),
    MOSRHistory(history_fpf_frequency=5),
]

In [ ]:
POPULATION_SIZE = 100
TOURNAMENT_SIZE = 3

logging.info(f'Running with POPULATION_SIZE {POPULATION_SIZE}')
logging.info(f'Running with TOURNAMENT_SIZE {TOURNAMENT_SIZE}')


sr = SymbolicRegressor(
    client_name='client',
    const_range=const_range,
    parsimony=.8,
    parsimony_decay=.85,  # Expected depth = parsimony / (1-parsimony_decay)
    population_size=POPULATION_SIZE,
    tournament_size=TOURNAMENT_SIZE,
    genetic_operators_frequency=genetic_operators_frequency,
)


In [ ]:
GENERATIONS = 100

sr.fit(
    data=train,
    val_data=test,
    features=features,
    operations=operations,
    fitness_functions=fitness_functions,
    generations_to_train=GENERATIONS,
    n_jobs=-1,
    stop_at_convergence=True,
    convergence_rolling_window=5,
    verbose=3  # The output could be very verbose. Consider using 0, 1, or 2 to reduce the verbosity
)

print('End')


### How to access the models and use them

You can access the models from ```sr.population: List``` or from ```sr.first_pareto_front: List```. The first one contains all the models generated during the evolution process, while the second one contains only the models that are in the Pareto front.

E.g., 
```python
model = sr.population[0]  # OR model = sr.first_pareto_front[0]
```

To see the model expression, use
```python
>>> str(model.program)  # It is only the string representation
```

Some relevant attributes of the model are
```python
>>> model.features_used
>>> model.fitness
>>> model.fitness_validation
```

To evaluate the model, use
```python
>>> model.evaluate(data)  # data should be a Dict, pd.Series or pd.DataFrame
```

In [ ]:
model = sr.population[0]

str(model.program)

In [ ]:
print(f"\nModel complexity:\n\t{model.complexity}")
print(f"\nModel fitness:\n\t{model.fitness}")
print(f"\nModel fitness_validation:\n\t{model.fitness_validation}")  # Is empty if no validation set is provided

In [ ]:
model.evaluate(data=data[features])